# 5 LLM 实战演练 2

本节使用 [智谱 AI](https://open.bigmodel.cn/) 的 GLM-4-Flash API。大部分代码与上一节类似。

In [2]:
ZHIPU_API_KEY = '' # 填入实际的 API Key

In [2]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key=ZHIPU_API_KEY)

In [3]:
from IPython.display import display, Markdown

response = client.chat.completions.create(
    model='glm-4-flash',
    messages=[dict(role='user', content='Introduce yourself.')]
)
display(Markdown(response.choices[0].message.content))

Hello! I'm ChatGLM, an AI assistant based on the language model GLM jointly trained by Tsinghua University KEG Lab and Zhipu AI Company. My goal is to help users solve their problems by answering their questions. Since I'm an AI, I don't have a personal experience, but I'll do my best to assist you.

In [4]:
from datasets import load_dataset

imdb = load_dataset('stanfordnlp/imdb')
imdb['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [5]:
def generate_prompt(text, shot=0):
    system = 'Please identify the sentiment of the following movie review. Output a single word, either "Positive" or "Negative", as your answer.'
    examples = [
        f"Example 1: {imdb['train'][0]['text']}\n\nAnswer: Negative\n\n",
        f"Example 2: {imdb['train'][12500]['text']}\n\nAnswer: Positive\n\n",
        f"Example 3: {imdb['train'][12499]['text']}\n\nAnswer: Negative\n\n",
        f"Example 4: {imdb['train'][24999]['text']}\n\nAnswer: Positive\n\n",
    ]
    system += ''.join(examples[:shot])
    user = f'Movie review: {text}\n\nAnswer: '
    return system, user

prompt = generate_prompt(imdb['test'][0]['text'], shot=2)
display(Markdown('\n\n'.join(prompt)))

Please identify the sentiment of the following movie review. Output a single word, either "Positive" or "Negative", as your answer.Example 1: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John Ford, had sex scenes in his films.<br /><br />I do commend the filmmakers for the fact that any sex shown in the film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn't have much of a plot.

Answer: Negative

Example 2: Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn't really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I'd have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.

Answer: Positive



Movie review: I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to always say "Gene Roddenberry's Earth..." otherwise people would not continue watching. Roddenberry's ashes must be turning in their orbit as this dull, cheap, poorly edited (watching it without advert breaks really brings this home) trudging Trabant of a show lumbers into space. Spoiler. So, kill off a main character. And then bring him back as another actor. Jeeez! Dallas all over again.

Answer: 

In [6]:
text = imdb['test'][:100]['text'] + imdb['test'][12500:12600]['text']
label = imdb['test'][:100]['label'] + imdb['test'][12500:12600]['label']

In [7]:
import tqdm

invalid_responses = []
for shot in [0, 2, 4]:
    correct, incorrect, invalid = 0, 0, 0
    for i in tqdm.tqdm(range(len(text))):
        system, user = generate_prompt(text[i], shot=shot)
        response = client.chat.completions.create(
            model='glm-4-flash',
            messages=[
                dict(role='system', content=system),
                dict(role='user', content=user)
            ]
        )
        response = response.choices[0].message.content.lower()
        is_positive = 'positive' in response
        is_negative = 'negative' in response
        if is_positive == is_negative:
            invalid += 1
            invalid_responses.append(response)
        elif is_positive == label[i]:
            correct += 1
        else:
            incorrect += 1
    print(f'{shot}-shot: {(correct + invalid / 2) / len(text) * 100:.2f}%, {correct=}, {incorrect=}, {invalid=}')

100%|██████████| 200/200 [01:34<00:00,  2.12it/s]


0-shot: 92.50%, correct=184, incorrect=14, invalid=2


100%|██████████| 200/200 [02:53<00:00,  1.15it/s]


2-shot: 96.50%, correct=193, incorrect=7, invalid=0


100%|██████████| 200/200 [02:18<00:00,  1.44it/s]

4-shot: 96.00%, correct=192, incorrect=8, invalid=0


除 zero-shot 准确率较低外，其他方面的结果与上一节基本一致。